In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
!pip install mtcnn

     |████████████████████████████████| 2.3MB 2.9MB/s 


In [ ]:
# import the necessary packages
from skimage.transform import rescale
from google.colab.patches import cv2_imshow
from scipy.spatial import distance as dist
from imutils import face_utils
from scipy import ndimage
from os import path
import numpy as np
import argparse
import imutils
import dlib
import cv2, random, sys, math, glob, os
# demonstrate face detection
from os import listdir
from PIL import Image
from numpy import asarray
from matplotlib import pyplot
from mtcnn.mtcnn import MTCNN
from google.colab import files


In [ ]:
#tìm vị trí mắt
def eye_aspect_ratio(eye):
	# compute the euclidean distances between the two sets of
	# vertical eye landmarks (x, y)-coordinates
	A = dist.euclidean(eye[1], eye[5])
	B = dist.euclidean(eye[2], eye[4])

	# compute the euclidean distance between the horizontal
	# eye landmark (x, y)-coordinates
	C = dist.euclidean(eye[0], eye[3])

	# compute the eye aspect ratio
	ear = (A + B) / (2.0 * C)

	# return the eye aspect ratio
	return ear

# extract a single face from a given photograph
def extract_face(resized):
	# load image from file
	image = resized
	# convert to RGB, if needed
	#image = image.convert('RGB')
	# convert to array
	pixels = asarray(image)
	#img_copy = asarray(img_copy)
	# create the detector, using default width
	detector = MTCNN()
	# detect faces in the image
	results = detector.detect_faces(pixels)
	if (len(results)) == 0:
		return 0
	# extract the bounding box from the first face
	x1, y1, width, height = results[0]['box']
	if width <120:
		return 0
	#print(results[0])
	# bug fix
	x1, y1 = abs(x1), abs(y1)
	x2, y2 = x1 + width, y1 + height
	# extract the face
	face = pixels[y1:y2, x1:x2]
	#mask = img_copy[y1:y2, x1:x2]
	# resize pixels to the model size
	image = Image.fromarray(face)
	face_array = asarray(image)
	return face_array

#kiểm tra khoảng cách 2 mắt
def khoangcach(resized):
	#cv2_imshow(img_copy)
	gray = cv2.cvtColor(img_copy, cv2.COLOR_BGR2GRAY)
	thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
		# Find contours and find centroid information on rotated rectangle
	cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

	tam=[]
	cnts = cnts[0] if len(cnts) == 2 else cnts[1]
	###mat phai
	centroid, dimensions, angle = cv2.minAreaRect(cnts[1])
	cv2.circle(img_copy, (int(centroid[0]), int(centroid[1])), 2, (36,255,12), -1)
	tam.append(centroid)
	###mat trai 
	centroid, dimensions, angle = cv2.minAreaRect(cnts[0])
	cv2.circle(img_copy, (int(centroid[0]), int(centroid[1])), 2, (36,255,12), -1)
	tam.append(centroid)
	#toa độ tâm 
	#print((int(tam[1][0]),int(tam[1][1])), (int(tam[0][0]),int(tam[0][1])))
	xtam2 = int(tam[1][0])
	xtam1 = int(tam[0][0])
	ytam2 = int(tam[1][1])
	ytam1 = int(tam[0][1])
	#print("tọa dộ 2 tâm mắt",(int(tam[1][0]),int(tam[1][1])), (int(tam[0][0]),int(tam[0][1])))
	#góc giữa đường thẳng và Ox
	angle = math.degrees(math.atan2(tam[1][1] - tam[0][1], tam[1][0] - tam[0][0]))
	print('góc giữa đường thẳng nối tầm mắt và trục Ox: ',angle)
	#khoảng cách giữa 2 tâm 
	distance = math.sqrt((xtam2-xtam1)**2 + (ytam2 - ytam1)**2)
	print('khoảng cách giữa tâm hai mắt: ',distance)
	return distance, angle

#Crop ảnh 160-160
def crop_center(pil_img, crop_width, crop_height):
    img_width, img_height = pil_img.size
    return pil_img.crop(((img_width - crop_width) // 2,
                         (img_height - crop_height) // 2,
                         (img_width + crop_width) // 2,
                         (img_height + crop_height) // 2))
		

In [ ]:
# construct the argument parser and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument('--shape-predictor', type=str,default='/content/drive/My Drive/Projects/Intern/shape_predictor_68_face_landmarks.dat',	help="path to facial landmark predictor")
ap.add_argument('--image',type=str, default='/content/drive/My Drive/Projects/Intern/database/Original',	help="path to input image")
args, _ = ap.parse_known_args()

# define two constants, one for the eye aspect ratio to indicate
# blink and then a second constant for the number of consecutive
# frames the eye must be below the threshold
EYE_AR_THRESH = 0.3
EYE_AR_CONSEC_FRAMES = 3
# initialize dlib's face detector (HOG-based) and then create
# the facial landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(args.shape_predictor)

# grab the indexes of the facial landmarks for the left and
# right eye, respectively
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]


In [ ]:
# tìm thư mục trong thư mục
format = [".jpg",".png",".jpeg",".tif", "JPG"]
path = '/content/drive/My Drive/Projects/Intern/database/Original/'
dem_image = 0
dem_err_rect =0 
dem_err_face = 0
for root, directories, files in os.walk(path, topdown=False):
  for name in files:
    if name.endswith(tuple(format)):
      print(str(root)+"/"+str(name))
      image = cv2.imread(str(root)+"/"+str(name))
      image = imutils.resize(image, width=500)
      width, height, _= np.shape(image)
      img = np.zeros((width, height,3),np.uint8)
      img_copy=img.copy()
      gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
      # detect faces in the grayscale image
      rects = detector(gray, 1)
      string = str(root)
      if len(rects) == 0:
        string=string.replace("/Original/", "/err_rects/")
        if not os.path.exists(string):
          os.makedirs(string)
        cv2.imwrite(string+"/"+str(os.path.basename(name)),gray)
        dem_err_rect = dem_err_rect + 1
        #print("anh duoc luu vao: ",string)
      else:
       # loop over the face detections
       for rect in rects:
               # determine the facial landmarks for the face region, then
               # convert the facial landmark (x, y)-coordinates to a NumPy
               # array
               shape = predictor(gray, rect)
               shape = face_utils.shape_to_np(shape)
 
               # extract the left and right eye coordinates, then use the
               # coordinates to compute the eye aspect ratio for both eyes
               leftEye = shape[lStart:lEnd]
               rightEye = shape[rStart:rEnd]


               # compute the convex hull for the left and right eye, then
               # visualize each of the eyes
               leftEyeHull = cv2.convexHull(leftEye)
               rightEyeHull = cv2.convexHull(rightEye)

               cv2.drawContours(img_copy, [leftEyeHull], -1, (0, 255, 0), 5)
               cv2.drawContours(img_copy, [rightEyeHull], -1, (0, 255, 0), 5)
       # show the output image with the face detections + facial landmarks
       #cv2_imshow(img_copy)
       gray = cv2.cvtColor(img_copy, cv2.COLOR_BGR2GRAY)
       thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
               # Find contours and find centroid information on rotated rectangle
       cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

       tam=[]
       cnts = cnts[0] if len(cnts) == 2 else cnts[1]
       ###mat phai
       centroid, dimensions, angle = cv2.minAreaRect(cnts[1])
       #cv2.circle(img_copy, (int(centroid[0]), int(centroid[1])), 2, (36,255,12), -1)
       tam.append(centroid)
       ###mat trai 
       centroid, dimensions, angle = cv2.minAreaRect(cnts[0])
       #cv2.circle(img_copy, (int(centroid[0]), int(centroid[1])), 2, (36,255,12), -1)
       tam.append(centroid)
       #toa độ tâm 
       print((int(tam[1][0]),int(tam[1][1])), (int(tam[0][0]),int(tam[0][1])))
       xtam2 = int(tam[1][0])
       xtam1 = int(tam[0][0])
       ytam2 = int(tam[1][1])
       ytam1 = int(tam[0][1])
       #đường thẳng nối 2 tâm
       line = cv2.line(img, (int(tam[1][0]),int(tam[1][1])), (int(tam[0][0]),int(tam[0][1])), (36,255,12), 1)
       type(line)
       #góc giữa đường thẳng và Ox
       angle = (math.degrees(math.atan2(tam[1][1] - tam[0][1], tam[1][0] - tam[0][0])))
       #xoay 1 góc angle
       image1 = ndimage.rotate(image, angle if -90<angle<90 else 180+angle)
       img_copy =  ndimage.rotate(img_copy, angle if -90<angle< 90 else 180+angle)
       #khoảng cách giữa 2 tâm 
       distance = math.sqrt((xtam2-xtam1)**2 + (ytam2 - ytam1)**2)
       
       print("khoảng cách 2 mắt trước khi điều chỉnh",distance)
       cv2_imshow(image1)
       #rescale ảnh
       rate = 80/distance
       width = int(image1.shape[1] * rate)
       height = int(image1.shape[0] * rate)
       dim = (width, height)
       resized = cv2.resize(image1, dim, interpolation = cv2.INTER_CUBIC)
       img_copy = cv2.resize(img_copy, dim, interpolation = cv2.INTER_CUBIC)
       
       #print("tọa dộ 2 tâm mắt",(int(tam[1][0]),int(tam[1][1])), (int(tam[0][0]),int(tam[0][1])))
       face= extract_face(resized)
       print('kích thước ảnh sau khi extract face',face.shape)
       cv2_imshow(face)
       #print(type(face))
       if type(face) == int:
         string = string.replace('/Original/', '/err_face/')
         if not os.path.exists(string):
           os.makedirs(string)
         cv2.imwrite(string+"/"+str(os.path.basename(name)),resized)
         dem_err_face = dem_err_face + 1
       else:
        #print("kích thước ảnh trước khi crop",face.shape)
        im = Image.fromarray(face)
        #mask = Image.fromarray(mask)
        im_new = crop_center(im, 160, 160)
        #mask_new = crop_center(mask, 160, 160)
        im_np = np.array(im_new)
        np_gray = cv2.cvtColor(im_np, cv2.COLOR_BGR2GRAY)
        #mask_np =  np.array(mask_new)
        #print("size anh sau khi điều chỉnh:", im_np.shape)
        print('kiểm tra kích thước ảnh lần cuối',np_gray.shape)
        cv2_imshow(np_gray)
        print("kiểm tra khoảng cách mắt lần cuối: ")
        kc = khoangcach(im_np)
        #lưu ảnh
        #cv2.imwrite('str(root)'+"/"+str(os.path.basename(name)), im_np)
        string=str(root)
        string=string.replace("/Original/", "/Result/")
        if not os.path.exists(string):
          os.makedirs(string)
        cv2.imwrite(string+"/"+str(os.path.basename(name)),np_gray)
        print("anh duoc luu vao: ",string)
        dem_image = dem_image +1
print("image", dem_image)
print('err_rect', dem_err_rect)
print('err_face', dem_err_face)
       
   #for name in directories:
     #print("tìm thấy thư mục: "+str(name),os.path.join(root, name))

Output hidden; open in https://colab.research.google.com to view.